In [ ]:
import pandas as pd
import requests
from google.colab import files

In [ ]:
import numpy as np
from datetime import datetime
from math import sqrt

# Upload and Preprocess the Dataset

In [ ]:
urls = [
    "http://pin13.net/car2go/data/2012-06.txt.gz"]
#    ,
#    "http://pin13.net/car2go/data/2012-07.txt.gz",
#    "http://pin13.net/car2go/data/2012-08.txt.gz",
#    "http://pin13.net/car2go/data/2012-09.txt.gz",
#    "http://pin13.net/car2go/data/2012-10.txt.gz",
#    "http://pin13.net/car2go/data/2012-12.txt.gz",
#    "http://pin13.net/car2go/data/2013-01.txt.gz",
#    "http://pin13.net/car2go/data/2013-02.txt.gz",
#    "http://pin13.net/car2go/data/2013-03.txt.gz",
#    "http://pin13.net/car2go/data/2013-04.txt.gz",
#    "http://pin13.net/car2go/data/2013-05.txt.gz",
#    "http://pin13.net/car2go/data/2013-06.txt.gz",
#    "http://pin13.net/car2go/data/2013-07.txt.gz",
#    "http://pin13.net/car2go/data/2013-08.txt.gz",
#    "http://pin13.net/car2go/data/2013-09.txt.gz",
#    "http://pin13.net/car2go/data/2013-10.txt.gz",
#    "http://pin13.net/car2go/data/2013-11.txt.gz"]

for url in urls:
    filename = url.split('/')[-1]

    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

#    print(f"File {filename} downloaded successfully!")

In [ ]:
#combine the dataset
filenames = ["2012-06.txt.gz"]

#filenames = [
    #"2012-06.txt.gz",
    #"2012-07.txt.gz"
   # ]

Data = pd.concat([pd.read_csv(f) for f in filenames],ignore_index=True)

In [ ]:
print(Data.shape)

(582304, 7)


In [ ]:
Data.head()

,date,license,latitude,longitude,fuel,interior,exterior
0,2012-06-01 20:58:56,191FRH,45.471205,-122.648474,100,true,true
1,2012-06-01 20:58:56,158FRH,45.558546,-122.709414,87,true,true
2,2012-06-01 20:58:56,192FRH,45.515036,-122.629782,54,true,true
3,2012-06-01 20:58:56,196FRH,45.554366,-122.671270,100,true,true
4,2012-06-01 20:58:56,197FRH,45.562482,-122.632860,48,true,true


Found there are leading spaces in the names of the columns and the values, then we get rid of the spaces

In [ ]:
Data.columns = [col.strip() for col in Data.columns]  #get rid of the leading space in the name of coulmns

In [ ]:
variable_names = Data.columns.tolist()
variable_names #check whether the names of columns are okay

['date', 'license', 'latitude', 'longitude', 'fuel', 'interior', 'exterior']

In [ ]:
Data = Data.applymap(lambda x: x.strip() if isinstance(x, str) else x) #get rid of the leading space in the values

Separate the date and time

In [ ]:
#sperate the Real date and time
Data['time'] = pd.to_datetime(Data['date']).dt.time
Data['date'] = pd.to_datetime(Data['date']).dt.date

In [ ]:
#adjust the order of columns
desired_order = ['license', 'date', 'time', 'latitude', 'longitude', 'fuel', 'interior', 'exterior',]
Data = Data[desired_order]

In [ ]:
Data.head(100)

,license,date,time,latitude,longitude,fuel,interior,exterior
0,191FRH,2012-06-01,20:58:56,45.471205,-122.648474,100,true,true
1,158FRH,2012-06-01,20:58:56,45.558546,-122.709414,87,true,true
2,192FRH,2012-06-01,20:58:56,45.515036,-122.629782,54,true,true
3,196FRH,2012-06-01,20:58:56,45.554366,-122.671270,100,true,true
4,197FRH,2012-06-01,20:58:56,45.562482,-122.632860,48,true,true
...,...,...,...,...,...,...,...,...
95,238FRH,2012-06-01,20:58:56,45.511086,-122.653625,78,true,true
96,237FRH,2012-06-01,20:58:56,45.516997,-122.593723,54,false,true
97,236FRH,2012-06-01,20:58:56,45.523839,-122.659648,66,true,true
98,216FRH,2012-06-01,20:58:56,45.518549,-122.627328,21,true,true


Change the results in interior and exterior to catergorical number

In [ ]:
# Convert true and false to 1 and 0, notice that is string not boolean
for col in ['interior','exterior']:
    Data[col] = Data[col].replace({'true': 1, 'false': 0})

In [ ]:
Data.head(100)

,license,date,time,latitude,longitude,fuel,interior,exterior
0,191FRH,2012-06-01,20:58:56,45.471205,-122.648474,100,1,1
1,158FRH,2012-06-01,20:58:56,45.558546,-122.709414,87,1,1
2,192FRH,2012-06-01,20:58:56,45.515036,-122.629782,54,1,1
3,196FRH,2012-06-01,20:58:56,45.554366,-122.671270,100,1,1
4,197FRH,2012-06-01,20:58:56,45.562482,-122.632860,48,1,1
...,...,...,...,...,...,...,...,...
95,238FRH,2012-06-01,20:58:56,45.511086,-122.653625,78,1,1
96,237FRH,2012-06-01,20:58:56,45.516997,-122.593723,54,0,1
97,236FRH,2012-06-01,20:58:56,45.523839,-122.659648,66,1,1
98,216FRH,2012-06-01,20:58:56,45.518549,-122.627328,21,1,1


# Locate the Renting Timepoint

For each vehicle, abs delta latitude >= 0.01 AND abs delta longitude >= 0.01 AND fuel decrease

Notes: The location/latitude/longitude would change which getting charged, so we should care whether the battery level decrease

In [ ]:
#to check the number of vehicles we are going to track
num_vehicle= Data['license'].nunique()
print(num_vehicle)  #The number of vehicles is not exactly as what they said 316 ALWAYS

246


In [ ]:
unique_vehicle =Data['license'].unique()

In [ ]:
#convert the numpy.ndarray to a list
unique_vehicle_list = unique_vehicle.tolist()
print(type(unique_vehicle_list))

<class 'list'>


Sort the records

In [ ]:
Data.sort_values(by=['license', 'date', 'time'], inplace=True) #sort the records as license, date and time

In [ ]:
Data.reset_index(drop=True, inplace=True)#set the new index

In [ ]:
#Data.to_csv('cleaned_data.csv', index=False)
#files.download('cleaned_data.csv')

Collect the filtered records

In [ ]:
def filter_general(group):
    group['diff_longitude'] = group['longitude'].diff().fillna(0)
    group['diff_latitude'] = group['latitude'].diff().fillna(0)

    group['diff_fuel'] = group['fuel'].diff().fillna(0)

    #combine the constraints
    filter = (group['diff_longitude'] != 0) & (group['diff_latitude'] != 0) & (group['diff_fuel'] <= 0)

    #Set the filter rows as 2, one previous row of each filter row as 1
    group['note'] = 0

    group.loc[filter, 'note'] = 2 #set filter rows as 2

    #get the filter rows' index
    filter_index = group.index[filter]

    #get the index of the shift back one row
    shiftback_index = filter_index - 1
    previous_sets = shiftback_index[shiftback_index >= group.index[0]]
    group.loc[previous_sets, 'note'] = 1 #set shift back one row as 1

    # only collect the note 1 or 2 row (either filtered rows or their previous rows)
    return group[group['note'] > 0]

filtered_df = Data.groupby('license', group_keys=False).apply(filter_general)
filtered_df = filtered_df.drop(['diff_longitude', 'diff_latitude', 'diff_fuel'], axis=1)

In [ ]:
print(filtered_df.head(30))

    license        date      time   latitude   longitude  fuel  interior  \
0    151FRH  2012-06-01  20:58:56  45.510180 -122.675216    33         1   
1    151FRH  2012-06-01  22:58:03  45.514420 -122.654003    33         1   
10   151FRH  2012-06-02  01:15:02  45.514420 -122.654003    33         1   
11   151FRH  2012-06-02  01:45:03  45.533480 -122.604074    30         0   
134  151FRH  2012-06-03  10:45:02  45.523880 -122.650722   100         1   
135  151FRH  2012-06-03  11:15:03  45.523861 -122.698368   100         1   
156  151FRH  2012-06-03  16:30:03  45.523861 -122.698368   100         1   
157  151FRH  2012-06-03  16:45:03  45.524908 -122.689948   100         1   
164  151FRH  2012-06-03  18:30:03  45.524908 -122.689948   100         1   
165  151FRH  2012-06-03  19:15:03  45.559033 -122.644220   100         1   
171  151FRH  2012-06-03  20:45:03  45.559033 -122.644220   100         1   
172  151FRH  2012-06-03  21:45:03  45.522510 -122.689915   100         1   
215  151FRH 

Deal with the "over-time" trips. <br>In the three or more consecutive rows, the location keep changed, that is, the vehicle did not stop within two recording timepoints. We assume that the vehicle kept moved, that is, it was only picked up once, ignoring the status of middle records

In [ ]:
#set one or more note==1 records and the only note==2 record as a group
#add the new column in the current dataset filtered_df
filtered_df['group_id'] = (filtered_df['note'] == 2).shift(1, fill_value=0).cumsum()

In [ ]:
print(filtered_df)

       license        date      time   latitude   longitude  fuel  interior  \
0       151FRH  2012-06-01  20:58:56  45.510180 -122.675216    33         1   
1       151FRH  2012-06-01  22:58:03  45.514420 -122.654003    33         1   
10      151FRH  2012-06-02  01:15:02  45.514420 -122.654003    33         1   
11      151FRH  2012-06-02  01:45:03  45.533480 -122.604074    30         0   
134     151FRH  2012-06-03  10:45:02  45.523880 -122.650722   100         1   
...        ...         ...       ...        ...         ...   ...       ...   
582195  511FSK  2012-06-29  14:00:08  45.530834 -122.679443    66         1   
582209  511FSK  2012-06-29  17:30:06  45.530834 -122.679443    66         1   
582210  511FSK  2012-06-29  18:30:05  45.561992 -122.694798    63         1   
582301  511FSK  2012-06-30  17:15:06  45.561992 -122.694798    63         1   
582302  511FSK  2012-06-30  17:45:07  45.518834 -122.683236    63         1   

        exterior  note group_id  
0              1 

In [ ]:
def define_group(group):
    # get the first row of note_value_1 rows in a group
    row_first_1 = group[group['note'] == 1].head(1)
    # get the only note_value_2 rows in a group
    row_only_2 = group[group['note'] == 2]
    # combine the two rows, that is, there will be fix mode, one note_value_1 row, and then one note_value_2 row in a group
    return pd.concat([row_first_1, row_only_2])

# get a new dataset, which get rid og the over-time trips
filtered_df2 = filtered_df.groupby('group_id', as_index=False).apply(define_group).reset_index(drop=True)

In [ ]:
print(filtered_df2.head(35))
#we can see that license "151FRH", date "2012-06-06", time point "08:00:04" has been got rid of
#which was between time points "07:15:07" and "09:15:03"

   license        date      time   latitude   longitude  fuel  interior  \
0   151FRH  2012-06-01  20:58:56  45.510180 -122.675216    33         1   
1   151FRH  2012-06-01  22:58:03  45.514420 -122.654003    33         1   
2   151FRH  2012-06-02  01:15:02  45.514420 -122.654003    33         1   
3   151FRH  2012-06-02  01:45:03  45.533480 -122.604074    30         0   
4   151FRH  2012-06-03  10:45:02  45.523880 -122.650722   100         1   
5   151FRH  2012-06-03  11:15:03  45.523861 -122.698368   100         1   
6   151FRH  2012-06-03  16:30:03  45.523861 -122.698368   100         1   
7   151FRH  2012-06-03  16:45:03  45.524908 -122.689948   100         1   
8   151FRH  2012-06-03  18:30:03  45.524908 -122.689948   100         1   
9   151FRH  2012-06-03  19:15:03  45.559033 -122.644220   100         1   
10  151FRH  2012-06-03  20:45:03  45.559033 -122.644220   100         1   
11  151FRH  2012-06-03  21:45:03  45.522510 -122.689915   100         1   
12  151FRH  2012-06-04  0

In [ ]:
# check how many rows of the new dataset
print(filtered_df2.shape)

(30474, 10)


Delete the overnight trips (Optional)

In [ ]:
#filter out the different date records in each group, creating a new dataset
filtered_df3 =filtered_df2.groupby('group_id').filter(lambda x: x['date'].nunique() == 1)

In [ ]:
print(filtered_df3.shape) #there ARE records filtered out, that is, there ARE overnight records.

(29446, 10)


In [ ]:
#check the filtered out rows
filtered_df3_complement = filtered_df2.groupby('group_id').filter(lambda x: x['date'].nunique() > 1)

print(filtered_df3_complement.head())

    license        date      time   latitude   longitude  fuel  interior  \
84   151FRH  2012-06-12  23:45:03  45.533820 -122.685521   100         1   
85   151FRH  2012-06-13  00:00:05  45.533903 -122.685473   100         1   
106  151FRH  2012-06-19  19:00:04  45.555451 -122.687037    78         1   
107  151FRH  2012-06-20  10:45:03  45.554385 -122.682160    78         1   
122  151FRH  2012-06-24  21:45:03  45.486210 -122.676724    63         1   

     exterior  note group_id  
84          0     1       42  
85          1     2       42  
106         1     1       53  
107         1     2       53  
122         1     1       61  


# Build a new dataset

Delete the groups which have long time gap
<br> Loss track for long, so that the records of distance and end time is not exact

create a newdataset for each group, which have columns, date, time, duration, distance, fuel, interior, exterior. The date, time is the same as the first row of each group, the duration is the gap between the time in two records, the distance is the square root of the sum of (longitude1-longitude2) square and (latitude1-latitude2) square. The fuel, interior and exterior is all the result of the first row.

Calculate the duration of the trip

In [ ]:
def duration_function(start_date, start_time, end_date, end_time):
    date = '%Y-%m-%d'
    time = '%H:%M:%S'
    # Combine date and time
    new_time = f'{date} {time}'

    # Parse the start and end date-times
    start = datetime.strptime(f"{start_date} {start_time}", new_time)
    end = datetime.strptime(f"{end_date} {end_time}", new_time)

    # Calculate the timedelta and return the duration in hours
    duration_trip = end - start
    return duration_trip.total_seconds() / 60  #per minutes

Caluculate the distance

In [ ]:
def distance_function(lat1, lon1, lat2, lon2):
    return sqrt((lon2 - lon1) ** 2 + (lat2 - lat1) ** 2)

Set new dataset

In [ ]:
new_df = []
for group_id, group in filtered_df2.groupby('group_id'):
    #if len(group) == 2: #to check whether there are only two records in a group, but it is not neccessary
        first_row = group.iloc[0]
        second_row = group.iloc[1]
        group_id=first_row['group_id']
        license = first_row['license']
        date = first_row['date'] #the date of pick-up is the date of the first row
        time = first_row['time'] #the time of pick up is the time of the first row
        trip_duration = duration_function(first_row['date'],  first_row['time'],  second_row['date'], second_row['time'])
        trip_distance = distance_function(first_row['latitude'], first_row['longitude'], second_row['latitude'], second_row['longitude'])
        longitude = first_row['longitude']
        latitude = first_row['latitude']
        fuel = first_row['fuel'] #the fuel(battery level) of pick-up is that of the first row
        interior = first_row['interior'] #also, maintain that of the first row
        exterior = first_row['exterior'] #maintain still

        # Create a new DataFrame for the group
        individual_group = pd.DataFrame({
            'license':[license],
            'date': [date],
            'time': [time],
            'longitude':[longitude],
            'latitude':[latitude],
            'duration': [trip_duration],
            'distance': [trip_distance],
            'fuel': [fuel],
            'interior': [interior],
            'exterior': [exterior]
        })

        new_df.append(individual_group)

#concat many individual groups to one group
final_df = pd.concat(new_df).reset_index(drop=True)
final_df.to_csv('final_df.csv', index=False)


In [ ]:
print(final_df.head(30))

   license        date      time   longitude   latitude    duration  distance  \
0   151FRH  2012-06-01  20:58:56 -122.675216  45.510180  119.116667  0.021632   
1   151FRH  2012-06-02  01:15:02 -122.654003  45.514420   30.016667  0.053443   
2   151FRH  2012-06-03  10:45:02 -122.650722  45.523880   30.016667  0.047646   
3   151FRH  2012-06-03  16:30:03 -122.698368  45.523861   15.000000  0.008484   
4   151FRH  2012-06-03  18:30:03 -122.689948  45.524908   45.000000  0.057058   
5   151FRH  2012-06-03  20:45:03 -122.644220  45.559033   60.000000  0.058498   
6   151FRH  2012-06-04  08:30:04 -122.689915  45.522510   30.000000  0.007095   
7   151FRH  2012-06-04  12:00:05 -122.683832  45.518857   29.966667  0.033603   
8   151FRH  2012-06-04  15:30:04 -122.652590  45.531230   30.000000  0.036753   
9   151FRH  2012-06-04  17:00:03 -122.688528  45.523530   30.000000  0.003919   
10  151FRH  2012-06-04  18:00:03 -122.685833  45.526375   30.016667  0.038800   
11  151FRH  2012-06-05  20:0

In [ ]:
#files.download('final_df.csv')

Deleted the long-duration(more than 2 hours) trip. They might fail to be recorded within the two hours, for example, the target vehicle has dropped off within one hour, but 24 hours later, another record has been uploaded.

In [ ]:
print(len(final_df))

15237


In [ ]:
final_df_1 = final_df[final_df['duration'] <= 120]

In [ ]:
print(len(final_df_1))

14248


# Find the nearby vehicles